In [77]:
import pandas as pd
import numpy as np
import math
import random

# Importing dataset

In [78]:
datasets = [
    "data/opsahl-ucsocial/out.opsahl-ucsocial",
    "data/soc-sign-bitcoinalpha/out.soc-sign-bitcoinalpha",
    "data/soc-sign-bitcoinotc/out.soc-sign-bitcoinotc"
]
skiprows = [[0, 1], [0], [0]]
current = 1
graph = pd.read_csv(
    datasets[current],
    names=["_from", "_to", "_weight", "_timestamp"],
    sep=" |\t",
    engine ='python',
    skiprows=skiprows[current]
)
# print(graph.dtypes)
# print(graph.head())

In [79]:
V = np.unique(graph["_from"]._append(graph["_to"]))
n = V.size
volume = graph["_timestamp"].size
print(f"{n = }, {volume = }")

n = 3783, volume = 24186


# Chapter 1

## Preparing static graph

In [80]:
graph_static = [set() for _ in range(n + 1)]
for _index, _from, _to, _weight, _timestamp in graph.itertuples():
    if _from == _to:
        continue
    graph_static[_from].add(_to)
    graph_static[_to].add(_from)


## Task 1.1

In [81]:
E_count = 0
for i in range(len(graph_static)):
    E_count += len(graph_static[i])
E_count //= 2
#print(E_count)

density = E_count * 2 / (n * (n - 1))
#print(density)

In [82]:
V_to_visit = set(V)
connectivity_components = []
while(V_to_visit):
    V_seen = set()
    queue = []
    for u in V_to_visit:
        queue.append(u)
        V_seen.add(u)
        break
    while queue:
        u = queue.pop()
        u_adjacent_to_visit = graph_static[u].difference(V_seen)
        for v in u_adjacent_to_visit:
            V_seen.add(v)
            queue.append(v)
    V_to_visit = V_to_visit.difference(V_seen)
    connectivity_components.append(V_seen)

sizes = list(map(lambda x: len(x), connectivity_components))
max_component_size = max(sizes)
max_connectivity_component_index = sizes.index(max_component_size)
proportion = max_component_size / len(V)
#print(f"{max_val = }, {max_connectivity_component_index = }, {proportion = }")


In [83]:
# 1.1
print("|V| = %i, |E| = %i, p = %f, number of components = %i, max component size = %i, max component proportion= % f"
      % (n, E_count, density, len(connectivity_components), max_component_size, proportion))


|V| = 3783, |E| = 14124, p = 0.001974, number of components = 5, max component size = 3775, max component proportion=  0.997885


## Task 1.2

In [84]:
component = list(connectivity_components[max_connectivity_component_index])
distances = [0] * (n + 2)
diameter = 0
radius = n + 1
distances_tmp = np.empty(n + 1, dtype=int)
if(n < 10000):
    for start in component:
        distances_tmp.fill(n + 1)
        V_visited = set()
        queue = [(start, 0)]
        queued = set([start])
        depth = 0
        u = start
        while queued:
            (u, depth) = queue.pop(0)
            queued.remove(u)
            V_visited.add(u)
            u_adjacent_to_visit = graph_static[u].difference(V_visited)
            for v in u_adjacent_to_visit:
                distances_tmp[v] = min(distances_tmp[v], depth + 1)
                if v not in queued:
                    queue.append((v, depth + 1))
                    queued.add(v)
        for d in range(start, n + 1):
            distances[distances_tmp[d]] += 1
        diameter = max(diameter, depth)
        radius = min(radius, depth)
        if not start % 10:
            # print(start, "/", n)
            pass
#print(distances)
all_dist = 0
for i in range(diameter + 1):
    all_dist += distances[i]
print(all_dist, "=", max_component_size * (max_component_size - 1) // 2,
      all_dist == max_component_size * (max_component_size - 1) // 2)


7123425 = 7123425 True


In [85]:
#percentile_90 = np.percentile(all_distances, 90)
percentile_90_ind = int(0.9 * all_dist)
percentile_90 = 0
ind_tmp = 0
for i in range(diameter + 1):
    ind_tmp += distances[i]
    if ind_tmp >= percentile_90_ind:
        percentile_90 = i
        break

In [86]:
def calculate_matrix(vertices):

    vertices_list = list(vertices)
    vertices_set = set(vertices)

    distances = []
    distance_matrix = dict()
    for u in vertices_list:
        distance_matrix[u] = dict()
        for v in vertices_list:
            if u != v:
                distance_matrix[u][v] = n + 1

    for start in vertices_list:
        V_to_calculate = set(vertices)
        V_to_calculate.discard(start)
        V_visited = set()
        queue = [(start, 0)]
        queued = set([start])
        max_depth = 0
        while queued and V_to_calculate:
            u, depth = queue.pop(0)
            max_depth = max(max_depth, depth)
            queued.discard(u)
            V_visited.add(u)
            u_adjacent_to_visit = graph_static[u].difference(V_visited)
            for v in u_adjacent_to_visit:
                if v in V_to_calculate:
                    distance = distance_matrix[start][v]
                    if depth + 1 < distance:
                        distance_matrix[start][v] = depth + 1
                        distance_matrix[v][start] = depth + 1
                        V_to_calculate.discard(v)
                if v not in queued:
                    queue.append((v, depth + 1))
                    queued.add(v)
    
    for u in vertices_list:
        for v in vertices_list:
            if u > v:
                distances.append(distance_matrix[u][v])
    eccentricities = dict()
    for u in distance_matrix:
        eccentricities[u] = max(distance_matrix[u].values())

    return (distance_matrix, eccentricities, distances)


In [87]:


component = list(connectivity_components[max_connectivity_component_index])

random_500_vertices = sorted(random.sample(component, 500))
random_1000_vertices = sorted(random.sample(component, 1000))

random_500_matrix, random_500_eccentricities, random_500_distances = calculate_matrix(
    random_500_vertices)
random_1000_matrix, random_1000_eccentricities, random_1000_distances = calculate_matrix(
    random_1000_vertices)

diameter_from_random_500 = max(random_500_eccentricities.values())
radius_from_random_500 = min(random_500_eccentricities.values())
percentile_90_from_random_500 = np.percentile(random_500_distances, 90)

print(f"{diameter_from_random_500 = }, {radius_from_random_500 = }, {percentile_90_from_random_500 = }")

diameter_from_random_1000 = max(random_1000_eccentricities.values())
radius_from_random_1000 = min(random_1000_eccentricities.values())
percentile_90_from_random_1000 = np.percentile(random_1000_distances, 90)

print(f"{diameter_from_random_1000 = }, {radius_from_random_1000 = }, {percentile_90_from_random_1000 = }")

                    

diameter_from_random_500 = 8, radius_from_random_500 = 5, percentile_90_from_random_500 = 5.0
diameter_from_random_1000 = 9, radius_from_random_1000 = 5, percentile_90_from_random_1000 = 4.0


In [88]:
def snowball(limit):
    vertices = {component[0], component[1]}
    while len(vertices) < limit:
        for v in vertices:
            if len(vertices) < limit:
                vertices = vertices.union(graph_static[v])
    return sorted(list(vertices))

snowball_500_vertices = snowball(500)
snowball_1000_vertices = snowball(1000)

snowball_500_matrix, snowball_500_eccentricities, snowball_500_distances = calculate_matrix(
    snowball_500_vertices)
snowball_1000_matrix, snowball_1000_eccentricities, snowball_1000_distances = calculate_matrix(
    snowball_1000_vertices)

diameter_from_snowball_500 = max(snowball_500_eccentricities.values())
radius_from_snowball_500 = min(snowball_500_eccentricities.values())
percentile_90_from_snowball_500 = np.percentile(snowball_500_distances, 90)

print(f"{diameter_from_snowball_500 = }, {radius_from_snowball_500 = }, {percentile_90_from_snowball_500 = }")

diameter_from_snowball_1000 = max(snowball_1000_eccentricities.values())
radius_from_snowball_1000 = min(snowball_1000_eccentricities.values())
percentile_90_from_snowball_1000 = np.percentile(snowball_1000_distances, 90)

print(f"{diameter_from_snowball_1000 = }, {radius_from_snowball_1000 = }, {percentile_90_from_snowball_1000 = }")


diameter_from_snowball_500 = 2, radius_from_snowball_500 = 1, percentile_90_from_snowball_500 = 2.0
diameter_from_snowball_1000 = 4, radius_from_snowball_1000 = 2, percentile_90_from_snowball_1000 = 3.0


In [89]:
# 1.2
print("diameter = %i, raduis = %i, percentile_90 = %i" 
      % (diameter, radius, percentile_90))

diameter = 10, raduis = 5, percentile_90 = 5


## Task 1.3

In [90]:
component = list(connectivity_components[max_connectivity_component_index])

Cl = dict()
for u in component:
    u_neighbors = graph_static[u]

    if len(u_neighbors) < 2:
        Cl[u] = 0
        continue

    Lu_doubled = 0
    for neighbor in u_neighbors:
        Lu_doubled += len(graph_static[neighbor].intersection(u_neighbors))
    Cl[u] = Lu_doubled / (len(u_neighbors) * (len(u_neighbors) - 1))

Cl_average = sum(Cl.values()) / len(Cl.values())

In [91]:
#1.3
print("Cl_average = %f" % (Cl_average))

Cl_average = 0.177003


## Task 1.4

In [92]:
R1 = 0
R2 = 0
R3 = 0
Re = 0
for i in range(1, n + 1):
    ki = len(graph_static[i])
    R1 += ki
    R2 += ki**2
    R3 += ki**3
    for j in graph_static[i]:
        kj = len(graph_static[j])
        Re += ki * kj
degree_associativity = (Re * R1 - R2**2) / (R3 * R1 - R2**2)

In [93]:
print("Degree associativity = %f" % (degree_associativity))

Degree associativity = -0.168516


# Chapter 2

## Static topological features

In [94]:
def get_static_topological_features():
    CN_static = {}
    AA_static = {}
    JC_static = {}
    PA_static = {}
    
    visited = set()
    for u in V:
        visited.add(u)
        for v in graph_static[u]:
            if v in visited:
                continue
            gamma_u = graph_static[u]
            gamma_v = graph_static[v]
            
            intersection_u_v = gamma_u.intersection(gamma_v)
            CN_static[(u, v)] = len(intersection_u_v)
            JC_static[(u, v)] = CN_static[(u, v)] / len(gamma_u.union(gamma_v))
            PA_static[(u, v)] = len(gamma_u) * len(gamma_v)
            AA_static[(u, v)] = sum([1.0 / np.log(len(graph_static[z])) for z in intersection_u_v])
    return CN_static, AA_static, JC_static, PA_static

CN_static, AA_static, JC_static, PA_static = get_static_topological_features()

## Node activity features

In [95]:
t_min = graph["_timestamp"].min()
t_max = graph["_timestamp"].max()
s = 2.0 / 3.0
t_s = t_min + (t_max - t_min) * s
l = 0.2

### Step 1: Temporal weighting

In [96]:
t_matrix = np.array([dict() for _ in range(n + 1)])
for _index, _from, _to, _weight, _timestamp in graph.itertuples():
    if _from == _to or _timestamp > t_s:
        continue
    
    if t_matrix[_from].get(_to) is not None: 
        t_matrix[_from][_to].append(_timestamp)
    else:
        t_matrix[_from][_to] = [_timestamp]
    
    if t_matrix[_to].get(_from) is not None: 
        t_matrix[_to][_from].append(_timestamp)
    else:
        t_matrix[_to][_from] = [_timestamp]
    
    
def get_temporal_weighting(l, t):
    time_var = (t - t_min) / (t_s - t_min)
    w_linear = l + (1 - l) * time_var
    w_exponential = l + (1 - l) * (np.exp(3 * time_var) - 1) / (np.exp(3) - 1)
    w_square_root = l + (1 - l) * np.sqrt(time_var)
    return [w_linear, w_exponential, w_square_root]

for vt in t_matrix:
    for v, times in vt.items():
        for i, t in enumerate(times):
            vt[v][i] = get_temporal_weighting(l, t)
    

### Step 2: Aggregation of node activity

In [97]:
class AggregationOfNodeActivity:  
    @staticmethod
    def zeroth_quantile(weights):
        return np.quantile(weights, 0)
    
    @staticmethod
    def first_quantile(weights):
        return np.quantile(weights, 0.25)
    
    @staticmethod
    def second_quantile(weights):
        return np.quantile(weights, 0.50)
    
    @staticmethod
    def third_quantile(weights):
        return np.quantile(weights, 0.75)
    
    @staticmethod
    def fourth_quantile(weights):
        return np.quantile(weights, 1)
    
    @staticmethod
    def get_sum(weights):
        return sum(weights)
    
    @staticmethod
    def get_mean(weights):
        return np.mean(weights)
    
    @staticmethod
    def aggregate(weights):
        return [
            AggregationOfNodeActivity.zeroth_quantile(weights),
            AggregationOfNodeActivity.first_quantile(weights),
            AggregationOfNodeActivity.second_quantile(weights),
            AggregationOfNodeActivity.third_quantile(weights),
            AggregationOfNodeActivity.fourth_quantile(weights),
            AggregationOfNodeActivity.get_sum(weights),
            AggregationOfNodeActivity.get_mean(weights)
        ]


node_activity_feature_graph = [list() for _ in range(len(t_matrix))]

for ind, vt in enumerate(t_matrix):
    if ind == 0 or len(vt) == 0:
        continue
    linears = []
    exponentials = []
    squares = []
    for v, weights in vt.items():
        if not linears:
            linears = [w[0] for w in weights]
            exponentials = [w[1] for w in weights]
            squares = [w[2] for w in weights]
            continue
        linears.extend([w[0] for w in weights])
        exponentials.extend([w[1] for w in weights])
        squares.extend([w[2] for w in weights])
    for weights in (linears, exponentials, squares):
        node_activity_feature_graph[ind].extend(
            AggregationOfNodeActivity.aggregate(weights)
        )

### Step 3: Combining node activity

In [106]:
class CombiningNodeActivity:
    @staticmethod
    def get_sum(a, b):
        return a + b
    
    @staticmethod
    def get_absolute_differrence(a, b):
        return abs(a - b)
    
    @staticmethod
    def get_minimum(a, b):
        return min(a, b)
    
    @staticmethod
    def get_maximum(a, b):
        return max(a, b)
    
    @staticmethod
    def combine(u_list, v_list):
        feature = []
        for a, b in zip(u_list, v_list):
            feature.append(CombiningNodeActivity.get_sum(a, b))
            feature.append(CombiningNodeActivity.get_absolute_differrence(a, b))
            feature.append(CombiningNodeActivity.get_minimum(a, b))
            feature.append(CombiningNodeActivity.get_maximum(a, b))
        return feature



In [107]:
pairs = set()

for u in range(1, n):
    u_adj = set(t_matrix[u].keys())
    for v in range(u + 1, n + 1):
        if v in u_adj:
            continue
        v_adj = set(t_matrix[v].keys())
        if v_adj.intersection(u_adj):
            pairs.add((u, v))


In [99]:
connecting = list()
non_connecting = list()

for u, v in pairs:
    if v in graph_static[u]:
        connecting.append((u, v))
    else:
        non_connecting.append((u, v))

print(len(connecting), len(non_connecting))

525 486481


In [112]:
m = 10
x_y = random.choices(connecting, k=m)
x_n = random.choices(non_connecting, k=m)

# all = train + test

x_all_pairs = list(x_y)
x_all_pairs.extend(x_n)

y_all = [1] * m
y_all.extend([0] * m)

x_all = []
features = dict.fromkeys(list(set(x_y).union(set(x_n))))
for u, v in features.keys():
    features[(u, v)] = CombiningNodeActivity.combine(
        node_activity_feature_graph[u], node_activity_feature_graph[v]
    )
for pair in x_all_pairs:
    x_all.append(features[pair])

x_train = []
x_test = []


In [113]:
counter1 = 0
counter = 0
print(t_min, t_s,t_max)
for _index, _from, _to, _weight, _timestamp in graph.itertuples():
    if _from == _to or _timestamp > t_s:
        continue
    counter += 1
print(counter)


1289192400 1398690000.0 1453438800
22115
